# Ensembles Boosting: Adaboost
We train a very simple Adaboost ensemble on a very simple dataset.  
The ensemble has just two estimators.

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from statistics import mean

In [ ]:
athlete = pd.read_csv('AthleteSelection.csv',index_col = 'Athlete')
athlete.head()

In [ ]:
names = athlete.index
feature_names = athlete.columns
y = athlete.pop('Selected').values
X = athlete.values

In [ ]:
X_train, X_test, y_train, y_test, n_tr, n_ts = train_test_split(X, y, names,
                                                                test_size=1/3, 
                                                                random_state = 4)

Train an AdaBoost ensemble with just two estimators

In [ ]:
abc = AdaBoostClassifier(n_estimators=2, algorithm = 'SAMME', random_state = 20)
abc.fit(X_train,y_train)
abc.estimator_weights_

The first estimator has 2/13 errors.

In [ ]:
abc.estimator_errors_, 2/13

The estimators are simple decision stumps so we can plot the thresholds in 2D space.

In [ ]:
e0 = abc.estimators_[0]
e1 = abc.estimators_[1]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
t0 = e0.tree_.threshold[0]
t1 = e1.tree_.threshold[0]
xlm = [2,9]
ylm = [1,10]
l1, l2 = [t0, t0], [1, 10]
color= ['red' if l == 'No' else 'blue' for l in y_train]
x1 = X_train[:,0]
x2 = X_train[:,1]
f = plt.figure(figsize=(8,5))

misx = [X_train[12,0],X_train[4,0]]
misy = [X_train[12,1],X_train[4,1]]
plt.scatter(misx, misy, color = 'orange', s = 150)


plt.scatter(x1,x2, color=color)
#plt.plot(l1,l2, color = 'b') 
plt.title("Athlete Selection")
plt.xlabel("Speed")
plt.ylabel("Agility")
plt.grid()
red_patch = mpatches.Patch(color='red', label='Not Selected')
blue_patch = mpatches.Patch(color='blue', label='Selected')
plt.legend(handles=[red_patch, blue_patch],loc=4)
for i, txt in enumerate(n_tr):
    plt.annotate(txt, (x1[i]+0.1, x2[i]), fontsize = 12)

plt.xlim(xlm)
plt.ylim(ylm)
plt.plot([t0,t0], ylm, color = 'g',linewidth=4)
plt.plot(xlm, [t1,t1], color = 'g')
f.savefig('Aths-2D.pdf')

e0 splits on feature 0, the speed feature. The split is at 5.

In [ ]:
e0.tree_.feature[0], e0.tree_.threshold[0]

e1 splits on feature 1, the Agility feature. The split is at 5.875.

In [ ]:
e1.tree_.feature[0], e1.tree_.threshold[0]

The two trees.

In [ ]:
from sklearn import tree
tree.plot_tree(e0,filled=True, feature_names=feature_names , label = 'none', impurity = False,
                rounded=True) 
plt.savefig('e0.pdf')

In [ ]:
tree.plot_tree(e1,filled=True, feature_names=feature_names , label = 'none', impurity = False,
                rounded=True) 
plt.savefig('e1.pdf')

### Sample weight update
Here we produce estimates of how we believe the instance weight update would work. 
In numpy `np.log()` is the natural log `ln`.

In [ ]:
s_weights0 = np.ones(13)*1/13
s_weights0

In [ ]:
epsilon0 = 2/13
alpha0 =0.5* np.log((1-epsilon0)/epsilon0)
alpha0

In [ ]:
upw = math.exp(alpha0)
upw

In [ ]:
downw = math.exp(-alpha0)
downw

In [ ]:
highw = 1/13*upw
highw

In [ ]:
s_weights1 = np.ones(13)*1/13*downw
s_weights1

Incorrectly classified examples at indexes 4 and 12.

In [ ]:
s_weights1[4] = highw
s_weights1[12] = highw
s_weights1

In [ ]:
y_pos = np.arange(len(s_weights1))
f = plt.figure(figsize=(5,1.5))
plt.bar(y_pos-0.15, s_weights0, align='center', width = 0.3, alpha=0.5, 
        label = "Est 0")
plt.bar(y_pos+0.15, s_weights1, align='center', color = 'r', width = 0.3, alpha=0.5,
       label = "Est 1")
plt.xticks(y_pos, n_tr)
plt.ylabel('Weight')
plt.xlabel('Sample')
plt.grid(axis = 'y')
plt.legend(loc = 'upper left')
plt.show()
f.savefig('weights.pdf', bbox_inches = 'tight')